In [1]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [2]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'E_no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

2 Ag_100_u2_RPBE_no001_CO_n1_d9.traj
6 Ag_100_u2_RPBE_no002_CO_n1_d9.traj
10 Ag_100_u2_RPBE_no003_CO_n1_d9.traj
16 Ag_110_u2_RPBE_no001_CO_n1_d9.traj
20 Ag_110_u2_RPBE_no002_CO_n1_d9.traj
24 Ag_110_u2_RPBE_no003_CO_n1_d9.traj
28 Ag_110_u2_RPBE_no004_CO_n1_d9.traj
34 Ag_111_u2_RPBE_no001_CO_n1_d9.traj
38 Ag_111_u2_RPBE_no002_CO_n1_d9.traj
42 Ag_111_u2_RPBE_no003_CO_n1_d9.traj
46 Ag_111_u2_RPBE_no004_CO_n1_d9.traj
52 Ag_211_u2_RPBE_no001_CO_n1_d9.traj
56 Ag_211_u2_RPBE_no002_CO_n1_d9.traj
60 Ag_211_u2_RPBE_no003_CO_n1_d9.traj
64 Ag_211_u2_RPBE_no004_CO_n1_d9.traj
68 Ag_211_u2_RPBE_no005_CO_n1_d9.traj
72 Ag_211_u2_RPBE_no006_CO_n1_d9.traj
76 Ag_211_u2_RPBE_no007_CO_n1_d9.traj
80 Ag_211_u2_RPBE_no008_CO_n1_d9.traj
84 Ag_211_u2_RPBE_no009_CO_n1_d9.traj
88 Ag_211_u2_RPBE_no010_CO_n1_d9.traj
94 Al_100_u2_RPBE_no001_CO_n1_d9.traj
98 Al_100_u2_RPBE_no002_CO_n1_d9.traj
102 Al_100_u2_RPBE_no003_CO_n1_d9.traj
108 Al_110_u2_RPBE_no001_CO_n1_d9.traj
112 Al_110_u2_RPBE_no002_CO_n1_d9.traj
116 Al_110_

In [3]:
### DELETE ###
for file in filenames:
    try:
        A = make_database(file)
        A.delete_from_database()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Ag_100_u2_RPBE_no001_CO_n1_d9.traj deleted
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Ag_100_u2_RPBE_no002_CO_n1_d9.traj deleted
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Ag_100_u2_RPBE_no003_CO_n1_d9.traj deleted
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Ag_110_u2_RPBE_no001_CO_n1_d9.traj deleted
-----------------------------------------------------------
Ag_110_u2_RPBE_no002_CO_n1_d9.traj
Ag_110_u2_RPBE_no002_CO_n1_d9.traj deleted
-----------------------------------------------------------
Ag_110_u2_RPBE_no003_CO_n1_d9.traj
Ag_110_u2_RPBE_no003_CO_n1_d9.traj deleted
-----------------------------------------------------------
Ag_110_u2_RPBE_no004_CO_n1_d9.traj
Ag_110_u2_RPBE_no004_CO_n1_d9.traj deleted
----------------------------------

Cu_100_u2_RPBE_no001_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE_no002_CO_n1_d8.traj
Cu_100_u2_RPBE_no002_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE_no003_CO_n1_d8.traj
Cu_100_u2_RPBE_no003_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_110_u2_RPBE_no001_CO_n1_d8.traj
Cu_110_u2_RPBE_no001_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_110_u2_RPBE_no002_CO_n1_d8.traj
Cu_110_u2_RPBE_no002_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_110_u2_RPBE_no003_CO_n1_d8.traj
Cu_110_u2_RPBE_no003_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_110_u2_RPBE_no004_CO_n1_d8.traj
Cu_110_u2_RPBE_no004_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_111_u2_RPBE_no001_CO_n1_d8.traj
Cu_111_u2_RPBE_no001_CO_n1_d8.traj

In [4]:
### ADD and UPDATE ENERGY ###
for file in filenames:
    try:
        A = make_database(file)
        A.add_to_database()
        A.update_Energy()
        A.update_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
One post: 5e66f17f69846abba1a74095

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
One post: 5e66f17f69846abba1a74097

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Not in database.
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
One post: 5e66f18069846abba1a74099

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_110_u2_

Al_110_u2_RPBE_no004_CO_n1_d9.traj
One post: 5e66f18b69846abba1a740cb

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Al_111_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
Al_111_u2_RPBE_no001_CO_n1_d9.traj
One post: 5e66f18c69846abba1a740cd

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Al_111_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
Al_111_u2_RPBE_no002_CO_n1_d9.traj
One post: 5e66f18d69846abba1a740cf

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Al_111_u2_RPBE_no003_CO_n1_d9.traj
Not in database.
Al_111_u2_RPBE_no003_CO_n1_d9.traj
One post: 5e66f18e69846abba1a740d1


Au_211_u2_RPBE_no002_CO_n1_d9.traj
One post: 5e66f19a69846abba1a74101

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Au_211_u2_RPBE_no003_CO_n1_d9.traj
Not in database.
Au_211_u2_RPBE_no003_CO_n1_d9.traj
One post: 5e66f19b69846abba1a74103

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Au_211_u2_RPBE_no004_CO_n1_d9.traj
Not in database.
Au_211_u2_RPBE_no004_CO_n1_d9.traj
One post: 5e66f19b69846abba1a74105

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Au_211_u2_RPBE_no005_CO_n1_d9.traj
Not in database.
Au_211_u2_RPBE_no005_CO_n1_d9.traj
One post: 5e66f19b69846abba1a74107


Cu_211_u2_RPBE_no004_CO_n1_d8.traj
One post: 5e66f1a969846abba1a74137

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Cu_211_u2_RPBE_no005_CO_n1_d8.traj
Not in database.
Cu_211_u2_RPBE_no005_CO_n1_d8.traj
One post: 5e66f1aa69846abba1a74139

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Cu_211_u2_RPBE_no006_CO_n1_d8.traj
Not in database.
Cu_211_u2_RPBE_no006_CO_n1_d8.traj
One post: 5e66f1aa69846abba1a7413b

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Cu_211_u2_RPBE_no007_CO_n1_d8.traj
Not in database.
Cu_211_u2_RPBE_no007_CO_n1_d8.traj
One post: 5e66f1ab69846abba1a7413d


Ni_211_u2_RPBE_no004_CO_n1_d8.traj
One post: 5e66f1c069846abba1a7416d

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ni_211_u2_RPBE_no006_CO_n1_d8.traj
Not in database.
Ni_211_u2_RPBE_no006_CO_n1_d8.traj
One post: 5e66f1c069846abba1a7416f

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ni_211_u2_RPBE_no007_CO_n1_d8.traj
Not in database.
Ni_211_u2_RPBE_no007_CO_n1_d8.traj
One post: 5e66f1c069846abba1a74171

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ni_211_u2_RPBE_no010_CO_n1_d8.traj
Not in database.
Ni_211_u2_RPBE_no010_CO_n1_d8.traj
One post: 5e66f1c169846abba1a74173


In [37]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['minimum_distance'], data['ads_dist2'], data['ads_dist3'])
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Ag_100_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Ag_100_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Ag_100_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Ag_110_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no002_CO_n1_d9.traj
Ag_110_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no003_CO_n1_d9.traj
Ag_110_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no004_CO_n1_d9.traj
Ag_110_u2_RPBE_no004_CO_n1_d9.traj 

-----------------------------------------------------------
Cu_111_u2_RPBE_no002_CO_n1_d8.traj
Cu_111_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_111_u2_RPBE_no003_CO_n1_d8.traj
Cu_111_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_111_u2_RPBE_no004_CO_n1_d8.traj
Cu_111_u2_RPBE_no004_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no001_CO_n1_d8.traj
Cu_211_u2_RPBE_no001_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no002_CO_n1_d8.traj
Cu_211_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no003_CO_n1_d8.traj
Cu_211_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no004_CO_n1_d8.traj
Cu_211_u2_RPBE_no004_CO_n1_d8.traj 

In [21]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [22]:
file = 'Cu_100_u2_RPBE_no006_CO_n4_d3.traj'
A = make_database(file)
A.check_database()

No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in init
No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in database
-----------------------------------------------------------
Cu_100_u2_RPBE_no006_CO_n4_d3.traj
Not in database.


In [47]:
query('Cu_110_u2_RPBE_no003_CO_n3_d3.traj','local')

Atoms(symbols='Cu16COCOCO', pbc=[True, True, False], cell=[7.354476072672618, 5.200399903061017, 23.900299927295762], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))